![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
# Start coding...
import pandas as pd
import numpy as np

# Read in csv
marketing = pd.read_csv("bank_marketing.csv")

# Split into the three tables
client = marketing[["client_id", "age", "job", "marital", 
                    "education", "credit_default", "mortgage"]]
campaign = marketing[["client_id", "number_contacts", "month", "day", 
               "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
economics = marketing[["client_id", "cons_price_idx", "euribor_three_months"]]

In [2]:
## Editing the client dataset
# Clean education column
client.loc[:, "education"] = client["education"].str.replace(".", "_")
client.loc[:, "education"] = client["education"].replace("unknown", np.nan)

In [3]:
# Clean job column
client.loc[:, "job"] = client["job"].str.replace(".", "_")

# Clean and convert client columns to bool data type
for col in ["credit_default", "mortgage"]:
    # Use .loc for assignment
    client.loc[:, col] = client[col].map({"yes": 1,
                                         "no": 0,
                                         "unknown": 0})
    client.loc[:, col] = client.loc[:, col].astype(bool)

In [6]:
client.isna().sum()

client_id            0
age                  0
job                  0
marital              0
education         1731
credit_default       0
mortgage             0
dtype: int64

### Campaign

In [10]:
# Change campaign_outcome to binary values
campaign.loc[:, "campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, "no": 0})

In [11]:
# Convert previous_outcome to binary values
campaign.loc[:, "previous_outcome"] = campaign["previous_outcome"].map({"success": 1, "failure": 0, "nonexistent": 0})

In [13]:
# Add year column
campaign.loc[:, "year"] = "2022"

# Convert day to string
campaign.loc[:, "day"] = campaign["day"].astype(str)

# Add last_contact_date column
campaign.loc[:, "last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

In [14]:
campaign.head()

,client_id,number_contacts,month,day,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,year,last_contact_date
0,0,1,may,13,261,0,0,NaN,2022,2022-may-13
1,1,1,may,19,149,0,0,NaN,2022,2022-may-19
2,2,1,may,23,226,0,0,NaN,2022,2022-may-23
3,3,1,may,27,151,0,0,NaN,2022,2022-may-27
4,4,1,may,3,307,0,0,NaN,2022,2022-may-3


In [15]:
# Convert to datetime
campaign.loc[:, "last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")

In [18]:
# Clean and convert outcome columns to bool
for col in ["campaign_outcome", "previous_outcome"]:
    campaign.loc[:, col] = campaign[col].astype(bool)

In [20]:
# Drop unneccessary columns
campaign = campaign.drop(columns=["month", "day", "year"])

In [21]:
campaign.head(2)

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0.0,1.0,261.0,0.0,False,True,2022-05-13 00:00:00
1,1.0,1.0,149.0,0.0,False,True,2022-05-19 00:00:00


In [26]:
campaign = campaign.dropna()

In [29]:
campaign['client_id'] = campaign['client_id'].astype('int')
campaign['number_contacts'] = campaign['number_contacts'].astype('int')

In [30]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   number_contacts             41188 non-null  int64  
 2   contact_duration            41188 non-null  float64
 3   previous_campaign_contacts  41188 non-null  float64
 4   previous_outcome            41188 non-null  bool   
 5   campaign_outcome            41188 non-null  bool   
 6   last_contact_date           41188 non-null  object 
dtypes: bool(2), float64(2), int64(2), object(1)
memory usage: 2.0+ MB
